In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from urllib.request import Request, urlopen


# Dataset for Solar Farms - Harmonised global datasets

Paper Link : https://www.nature.com/articles/s41597-020-0469-8 <br>
Dataset link: https://figshare.com/articles/dataset/Harmonised_global_datasets_of_wind_and_solar_farm_locations_and_power/11310269/2

*"There are no restrictions on the use of these data. The data should be cited as follows: Dunnett, S. Harmonised global datasets of wind and solar farm locations and power. figshare. Dataset. https://doi.org/10.6084/m9.figshare.11310269.v1 (2020)"*

In [8]:
solar_farms_data_path = 'global_solar_2020.csv'
solar_farms_all = pd.read_csv(solar_farms_data_path)

solar_farms_US  = solar_farms_all[solar_farms_all['GID_0']=='USA']
solar_farms_US.describe()

,X,Y,sol_id,panels,panel.area,landscape.area,water,urban,power
count,3.949000e+03,3.949000e+03,3949.000000,3949.000000,3949.000000,3949.000000,3949.000000,3949.000000,1127.000000
mean,-8.386997e+06,4.687226e+06,14292.681438,153.322867,0.128300,0.244227,0.000760,0.431755,16.367508
std,1.695486e+06,4.956819e+05,6779.123122,2331.445836,0.745280,1.294819,0.027555,0.495383,38.284127
min,-1.456521e+07,2.554883e+06,2.000000,1.000000,0.000003,0.000000,0.000000,0.000000,0.841058
25%,-1.000998e+07,4.408598e+06,10949.000000,3.000000,0.000141,0.000248,0.000000,0.000000,2.288516
50%,-8.680626e+06,4.637474e+06,11954.000000,6.000000,0.001866,0.009032,0.000000,0.000000,4.343822
75%,-6.748695e+06,5.078871e+06,20089.000000,16.000000,0.032051,0.069371,0.000000,1.000000,13.590260
max,-5.524927e+06,7.472106e+06,34238.000000,110246.000000,22.289277,39.411116,1.000000,1.000000,551.823434


# Filter required data


In [28]:
min_panel_area = 1.0
solar_farms_US = solar_farms_US[solar_farms_US['panel.area']>min_panel_area]

## The (X,Y) above are in "Eckert IV equal-area projection (EPSG:54012)" format.
I couldn't find a direct function to convert these into the normal latitude, longitude format that we use. So I used an API from a website that converts it (https://epsg.io/transform#s_srs=3857&t_srs=4326&x=NaN&y=NaN). You can put this coordinates in google maps to get the image of the solar site.

A direct funtion might be possible somehow from https://pyproj4.github.io/pyproj/stable/ but I am not sure how to do it.

In [32]:
# NOTE : Order of (X,Y) is flipped at the end because google maps needs it that way

def get_google_coordinates(X,Y):
    
    url = 'http://epsg.io/trans?x='+str(X)+'&y='+str(Y)+'&z=0&s_srs='+str(54012)+'&t_srs='+str(4326)+'&callback=jsonpFunction'
    
    req = Request(
    url=url,#'http://www.cmegroup.com/trading/products/#sortField=oi&sortAsc=false&venues=3&page=1&cleared=1&group=1', 
    headers={'User-Agent': 'Mozilla/5.0'}
    )
    webpage = urlopen(req).read()
    webpage_str = webpage.decode()
    x = float(webpage_str.split('"x": "')[1].split('", "y":')[0])
    y = float(webpage_str.split('"y": "')[1].split('", "z":')[0])
    #print("Order of (x,y) if flipped for input into google maps")
    return y,x

In [35]:
ind = 10

X_old = np.array(solar_farms_US['X'])[ind]
Y_old = np.array(solar_farms_US['Y'])[ind]

X_new,Y_new = get_google_coordinates(X_old, Y_old)
print(X_new,Y_new)

34.82641469953629 -118.40779822579395
